In [ ]:
import numpy as np
import pandas as pd


from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
from datetime import datetime



In [ ]:
business_df = pd.read_json("./data/yelp_academic_dataset_business.json", lines=True)

In [ ]:
business_df.shape

In [ ]:
business_df.head()

In [ ]:
business_df["state"].value_counts()

In [ ]:
pop_df = pd.read_csv("./data/cbsa-est2019-alldata.csv", encoding='latin1')

In [ ]:
pop_df.head()

In [ ]:
pop_df.shape

In [ ]:
search_strings = ['Philadelphia-Camden', 'Tampa-St.', 'Nashville', 'Indianapolis', 'St. Louis, MO-IL', 'New Orleans', 'Tucson', 'Reno, NV', 'Santa Maria-Santa Barbara', 'Boise City']
pattern = '|'.join(search_strings)

# Filter for exact matches
df_metros = pop_df[pop_df['NAME'].str.contains(pattern, case=False, na=False)]

In [ ]:
df_metros

In [ ]:
# take just the needed columns
df_metros = df_metros.iloc[:, 3:17]

In [ ]:
df_metros.columns[3:14]

In [ ]:
# rename the columns, stripping just the year
columns_to_rename = df_metros.columns[3:14]

# Create a mapping for just those columns
rename_dict = {col: col[-4:] for col in columns_to_rename}
df_metros = df_metros.rename(columns=rename_dict)

#rename the  city names column
df_metros = df_metros.rename(columns={'NAME': 'metro'})

In [ ]:
# drop more unnecessary columns
df_metros.drop(columns=['LSAD', 'CENSUS2010POP'], inplace=True)

In [ ]:
# change the name of the metro areas
df_metros['metro'] = df_metros['metro'].str.split(r'[,\s-]').str[0]

In [ ]:
df_metros['metro'] = df_metros['metro'].replace({'New': 'New Orleans', 'St.': 'St. Louis', 'Santa': 'Santa Barbara'})

In [ ]:
df_metros

In [ ]:
# melt the table
df_long = df_metros.melt(id_vars='metro', var_name='Year', value_name='Population')
print(df_long)

# Loading the population data to SQL

In [ ]:
# getting API and DB credentials - Alternative 1: dotenv_values()

config = dotenv_values()
 
pg_user = config['POSTGRES_USER'] # align the key labels with your .env file
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

In [ ]:
# updating the url
url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

# creating the engine
engine = create_engine(url, echo=False)

In [ ]:
engine.url # checking the url (password is hidden)

In [ ]:
# writing population dataframe to SQL
df_long.to_sql(name = 'population_by_metro', 
                       con = engine, 
                       schema = pg_schema, # pandas is allowing to specify, in which schema the table shall be created
                       if_exists='replace', 
                       index=False
                      )